In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.nn.functional as F
from scipy import ndimage
import nibabel as nib
import sys
import os
import numpy as np

import spinecnn_CONFIG
from utils.file_process import load_lines
from datasets.brains18 import BrainS18Dataset
from model_zoo.builder import generate_model
from evaluation.testing import test_3d_cnn, calculate_segmentation_dice
from evaluation.visualization import plot_3d_masks_marching_cubes

In [ ]:
NO_CUDA = spinecnn_CONFIG.NO_CUDA
DATA_FOLDER = spinecnn_CONFIG.DATA_FOLDER
CHECKPOINT_PATH = spinecnn_CONFIG.CHECKPOINT_PATH
TESTING_DATA_LIST = spinecnn_CONFIG.TESTING_DATA_LIST
NUM_CLASSES = spinecnn_CONFIG.NUM_CLASSES
MODEL_TYPE = spinecnn_CONFIG.MODEL_TYPE
MODEL_DEPTH = spinecnn_CONFIG.MODEL_DEPTH
INPUT_W = spinecnn_CONFIG.INPUT_W
INPUT_H = spinecnn_CONFIG.INPUT_H
INPUT_D = spinecnn_CONFIG.INPUT_D
RESNET_SHORTCUT = spinecnn_CONFIG.RESNET_SHORTCUT
PRETRAIN_PATH = spinecnn_CONFIG.PRETRAIN_PATH
GPU_ID = spinecnn_CONFIG.GPU_ID
NEW_LAYER_NAMES = spinecnn_CONFIG.NEW_LAYER_NAMES

In [ ]:
if not NO_CUDA:
    print("Using GPU for testing")
else:
    print("Using CPU for testing")

# getting model
training_checkpoint_path = torch.load(CHECKPOINT_PATH) # selected training checkpoint
model, parameters = generate_model(MODEL_TYPE, MODEL_DEPTH,
            INPUT_W, INPUT_H, INPUT_D, NUM_CLASSES,
            NEW_LAYER_NAMES, PRETRAIN_PATH,
            RESNET_SHORTCUT, NO_CUDA, GPU_ID,
            model_phase='test')
model.load_state_dict(training_checkpoint_path['state_dict'])

In [ ]:
# data tensor
testing_data = BrainS18Dataset(DATA_FOLDER, TESTING_DATA_LIST, INPUT_D, INPUT_H, INPUT_W, phase='test')
data_loader = DataLoader(testing_data, batch_size=1, shuffle=False, num_workers=1, pin_memory=False) # temporary

In [ ]:
# testing
test_img_names = [info.split(" ")[0] for info in load_lines(TESTING_DATA_LIST)]
output_masks = test_3d_cnn(data_loader, model, test_img_names)
np.save('output_mask.npy', output_masks[0])  # save output masks for further evaluation

In [ ]:
# evaluation: calculate dice 
label_names = [info.split(" ")[1] for info in load_lines(TESTING_DATA_LIST)]
Nimg = len(label_names)
dices = np.zeros([Nimg, NUM_CLASSES])
for idx in range(Nimg):
    label = nib.load(os.path.join(DATA_FOLDER, label_names[idx]))
    label = label.get_fdata()
    dices[idx, :] = calculate_segmentation_dice(output_masks[idx], label, range(NUM_CLASSES))

# print result
for idx in range(0, NUM_CLASSES):
    mean_dice_per_task = np.mean(dices[:, idx])
    print('mean dice for class-{} is {}'.format(idx, mean_dice_per_task))   

In [ ]:
# visualization
mask_to_view = output_masks[0]
plot_3d_masks_marching_cubes(mask_to_view)